In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
eQTL_dir = '/CIMA/Result/eQTL_tensorqtl/'
eQTL_temp_dir = '/CIMA/Data/SMR_eQTL_besd/result_for_besd/'
eQTL_output_dir = '/CIMA/Data/SMR_eQTL_besd/besd/'
smr = '/media/scPBMC1_AnalysisDisk1/huangzhuoli/Script_HPC/software_gaoyue/SMR/smr-1.3.1-linux-x86_64/smr-1.3.1'

In [3]:
CT_list = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scRNA.csv')

In [8]:
CT_list['final_annotation'].to_csv('/CIMA/Script/all_69_CT.txt',sep='\t',header=False,index=False)

In [4]:
#update_esi
bim = pd.read_csv('/CIMA/genetics/qc/10.maf01.bim', header=None, sep='\t')
bim.columns = ['chr','variant_id','dis','pos','A1','A2']
bed_df = pd.read_csv('/media/scPBMC1_AnalysisDisk1/huangzhuoli/hw5_backup/gaoyue/health/Data/SMR/tss.strand.csv')

In [5]:
for celltype in CT_list['final_annotation'].to_list():
    print('processing_'+celltype)
    eGene = pd.read_csv('/CIMA/Result/20250108_cis_eQTL_studywise_sig.csv')
    eGene = eGene[eGene['celltype'] == celltype]
    eGene = eGene['phenotype_id'].unique()
    
    parquet_files = [f for f in os.listdir(f'{eQTL_dir}{celltype}') if f.endswith('.parquet')]

    # 初始化一个空的 DataFrame 来存储结果
    eQTL_match = pd.DataFrame()

    # 循环遍历每个.parquet文件
    for file in parquet_files:
        # 读取.parquet文件为 DataFrame
        file_path = os.path.join(f'{eQTL_dir}{celltype}', file)
        eQTL_result = pd.read_parquet(file_path)    
        # 过滤数据并合并
        eQTL_result_match = eQTL_result[eQTL_result['phenotype_id'].isin(eGene)]
        eQTL_match = pd.concat([eQTL_match, eQTL_result_match])
    
    #make besd
    eQTL_match_besd = eQTL_match.loc[:,['variant_id','phenotype_id','slope','slope','pval_nominal','pval_nominal']]
    eQTL_match_besd.columns = ['SNP','gene','beta','t-stat','p-value','FDR']
    eQTL_match_besd ['t-stat'] = 'NA'
    eQTL_match_besd ['FDR'] = 'NA'
    eQTL_match_besd.to_csv(f'{eQTL_temp_dir}/{celltype}_eQTL_for_besd.txt', index=False, sep='\t')

    #linux command
    eqtl = f'{eQTL_temp_dir}/{celltype}_eQTL_for_besd.txt'
    gi = f'{smr} --eqtl-summary {eqtl} --matrix-eqtl-format --make-besd --out {eQTL_output_dir}{celltype}'
    os.system(gi)
    
    print('updating_'+celltype)
    esi = pd.read_csv(f'{eQTL_output_dir}{celltype}.esi', sep='\t', header=None)
    esi.columns = ['chr','variant_id','dis','pos','A1','A2','af']
    esi = esi[['variant_id']]
    esi = pd.merge(esi, bim.loc[:,['chr','variant_id','dis','pos','A1','A2']], left_on='variant_id', right_on='variant_id')

    af_df = eQTL_match.loc[:,['variant_id','af']]
    af_df_dup = af_df.duplicated(subset='variant_id', keep='first')
    af_df = af_df.loc[~af_df_dup]

    esi = pd.merge(esi, af_df, left_on='variant_id', right_on='variant_id')
    esi = esi.loc[:,['chr','variant_id','dis','pos','A1','A2','af']]
    esi.to_csv(f'{eQTL_output_dir}{celltype}_update.esi', sep='\t', header=None, index=False)

    #linux command
    gi = f'{smr} --beqtl-summary {eQTL_output_dir}{celltype} --update-esi {eQTL_output_dir}{celltype}_update.esi'
    os.system(gi)

    #updata_epi
    epi = pd.read_csv(f'{eQTL_output_dir}{celltype}.epi', sep='\t', header=None)
    epi.columns = ['chr','prob','dis','pos','gene_id','strand']
    epi = epi[['prob']]
    epi = pd.merge(epi, bed_df, left_on='prob',right_on='prob')
    epi.loc[:,'dis'] = 0
    epi = epi.loc[:,['chr','prob','dis','pos','gene_id','strand']]
    epi.to_csv(f'{eQTL_output_dir}{celltype}_update.epi', index=False, header=None, sep='\t')

    #linux command
    gi = f'{smr} --beqtl-summary {eQTL_output_dir}{celltype} --update-epi {eQTL_output_dir}{celltype}_update.epi'
    os.system(gi)

processing_Atypical_Bm_ITGAX
*******************************************************************
* Summary-data-based Mendelian Randomization (SMR)
* Version 1.3.1
* Build at Sep 21 2022 12:13:19, by GCC 8.3
* (C) 2015 Futao Zhang, Zhihong Zhu and Jian Yang
* The University of Queensland
* MIT License
*******************************************************************
Analysis started: 11:32:32,Sun Jan 19,2025

Options:
--eqtl-summary /CIMA/Data/SMR_eQTL_besd/result_for_besd//Atypical_Bm_ITGAX_eQTL_for_besd.txt
--matrix-eqtl-format 
--make-besd 
--out /CIMA/Data/SMR_eQTL_besd/besd/Atypical_Bm_ITGAX

Reading eQTL summary data from /CIMA/Data/SMR_eQTL_besd/result_for_besd//Atypical_Bm_ITGAX_eQTL_for_besd.txt ...
2201205 rows to be included from /CIMA/Data/SMR_eQTL_besd/result_for_besd//Atypical_Bm_ITGAX_eQTL_for_besd.txt.

Generating the .epi file...
617 probes have been saved in the file /CIMA/Data/SMR_eQTL_besd/besd/Atypical_Bm_ITGAX.epi.

Generating the .esi file...
1432053 SNPs have 